In [1]:
from premise import *
from premise.utils import delete_all_pickles
from datapackage import Package
import bw2data, bw2io, bw2calc
from unfold import Unfold
bw2data.projects.set_current("ecoinvent-3.11-cutoff")

In [4]:
scenarios = [
    {"model": "remind", "pathway":"SSP2-NPi", "year": 2050},
    #{"model": "image", "pathway":"SSP2-RCP19", "year": 2048},
]


ndb = NewDatabase(
        scenarios = scenarios,
        source_db=db_maps[ei][model] if source == "brightway" else None,
        source_type=source,
        source_file_path=f"/Users/romain/Documents/ecoinvent {ei}_cutoff_ecoSpold02/datasets" if source == "ecospold" else None, # <-- this is NEW
        source_version=ei,
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        system_model=model,
        system_args=combinations[0],
        biosphere_name="ecoinvent-3.10-biosphere" if ei == "3.10" else "biosphere3",
        #additional_inventories= [ # <-- this is NEW
        #    {"filepath": "/Users/romain/GitHub/premise/premise/data/additional_inventories/lci-PV.xlsx", "ecoinvent version": "3.7"}, # <-- this is NEW
        #] # <-- this is NEW
)


Inventory cache cleared!
premise v.(2, 1, 1, 'dev4')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+-------------------------------------------------

Processing scenarios: 100%|█████████████| 2/2 [02:56<00:00, 88.28s/it]

Done!



In [6]:
import bw2data, bw2io, bw2calc
bw2data.projects.set_current("ei38_2")

In [4]:
[(f["name"], f["code"]) for f in bw2data.Database("biosphere3") if f["name"] == "Fish, pelagic, in ocean"]

[('Fish, pelagic, in ocean', 'd22719d0-8afd-45d3-b9eb-222112f84cc7')]